<a href="https://colab.research.google.com/github/Alirs2002/skimLit-200k-RTC-PubMed/blob/main/skimlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this project,I am going to classify pubMed 200k RCT dataset's sentences,into 5 groups

1-objective

2-methods

3-results

4-conclusions

5-background

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

now,lets get the data

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git


Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (39/39), 177.08 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.


In [3]:
!ls

pubmed-rct  sample_data


In [4]:
!cd pubmed-rct/

In [5]:
!ls pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign

dev.txt  test.txt  train.zip


In [6]:
data_dir = "pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign"

In [7]:
import os

file_names = [data_dir + file_name for file_name in os.listdir(data_dir)]
print(file_names)

['pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signtrain.zip', 'pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signdev.txt', 'pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_signtest.txt']


In [8]:
!pwd

/content


In [9]:
import zipfile

zipfile = zipfile.ZipFile("./pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.zip")
zipfile.extractall()

In [10]:
!ls

pubmed-rct  sample_data  train.txt


In [11]:
def read_lines(filename):
  listt = []
  with open(filename,"r") as f:
    for line in f:
      listt.append(line)

  return listt


In [12]:
lines = read_lines("train.txt")
print(len(lines))

2593169


In [13]:
print(lines[:14])

['###24491034\n', 'BACKGROUND\tThe emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .\n', 'BACKGROUND\tThis paper describes the design and evaluation of Positive Outlook , an online program aiming to enhance the self-management skills of gay men living with HIV .\n', 'METHODS\tThis study is designed as a randomised controlled trial in which men living with HIV in Australia will be assigned to either an intervention group or usual care control group .\n', "METHODS\tThe intervention group will participate in the online group program ` Positive Outlook ' .\n", 'METHODS\tThe program is based on self-efficacy theory and uses a self-management approach to enhance skills , confidence and abilities to manage the psychosocial issues associated with HIV in daily life .\n', 'METHODS\tParticipants will access the program for a m

now,it is time to create a preprocessing function

In [14]:
def preprocess(filename):

  input_lines = read_lines(filename)

  abstract_lines = ""
  abstract_samples = []


  for line in input_lines:
    if line.startswith("###"):
      abstract_id = line
      abstract_lines = ""
    elif line.isspace():
      abstract_line_split = abstract_lines.splitlines()

      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {}
        target_text_split = abstract_line.split("\t")
        line_data["target"] = target_text_split[0]
        line_data["text"] = target_text_split[1].lower()
        line_data["line_number"] = abstract_line_number
        line_data["total_lines"] = len(abstract_line_split) - 1
        abstract_samples.append(line_data)

    else:
      abstract_lines += line
  return abstract_samples

In [15]:
test_sample_data = preprocess("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/test.txt")
train_sample_data = preprocess("/content/train.txt")
validation_sample_data = preprocess("/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign/dev.txt")


In [16]:
train_sample_data[10:15]

[{'target': 'BACKGROUND',
  'text': 'actrn@ .',
  'line_number': 10,
  'total_lines': 10},
 {'target': 'BACKGROUND',
  'text': 'the aim of this study was to evaluate the efficacy , safety and complications of orbital steroid injection versus oral steroid therapy in the management of thyroid-related ophthalmopathy .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients suffering from thyroid ophthalmopathy were included in this study .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'patients were randomized into two groups : group i included @ patients treated with oral prednisolone and group ii included @ patients treated with peribulbar triamcinolone orbital injection .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'only @ patients in both groups ( @ female and @ male ) completed the study .',
  'line_number': 3,
  'total_lines': 11}]

as you can see,our preprocessing function is working great

In [ ]:
# now,lets build a pandas dataframe,for better preformance

In [17]:
len(train_sample_data),len(test_sample_data),len(validation_sample_data)

(2211861, 29493, 28932)

In [18]:
train_data = pd.DataFrame(train_sample_data)
test_data = pd.DataFrame(test_sample_data)
val_data = pd.DataFrame(validation_sample_data)

In [19]:
train_data

,target,text,line_number,total_lines
0,BACKGROUND,the emergence of hiv as a chronic condition me...,0,10
1,BACKGROUND,this paper describes the design and evaluation...,1,10
2,METHODS,this study is designed as a randomised control...,2,10
3,METHODS,the intervention group will participate in the...,3,10
4,METHODS,the program is based on self-efficacy theory a...,4,10
...,...,...,...,...
2211856,METHODS,"after crossover , the wait-list group received...",5,9
2211857,METHODS,performance was assessed by comparison to amer...,6,9
2211858,RESULTS,performance improved significantly after simul...,7,9
2211859,RESULTS,no improvement was detected as a function of c...,8,9


now, i think we should make our own datasets of train,test and validation sentences and train,test and validation labels

In [20]:
# craeting sentences
train_sentence = train_data["text"]
test_sentence = test_data["text"]
val_sentence = val_data["text"]


# creating labels
train_label = train_data["target"]
test_label = test_data["target"]
val_label = val_data["target"]


In [21]:
lengths = np.mean([len(sentence.split()) for sentence in train_sentence])

In [22]:
lengths

26.229355280462922

In [23]:
train_sentence[3],len(train_sentence[3])

("the intervention group will participate in the online group program ` positive outlook ' .",
 90)

In [24]:
sentence_lengths = [len(sentence.split()) for sentence in train_sentence]
sentence_lengths[:10]

[36, 27, 31, 15, 31, 17, 19, 25, 25, 26]

as we can see,98 percent of our sentences,are 67 or shorter

In [25]:
np.percentile(sentence_lengths,96)

57.0

as you can see,the average length of a sentence in our dataset is 26,so our text vectorizer,should be capable of vectorizing this big sentences

our next step is to vectorize our sentences

in the article,it is written that the vocabulary length of this pubmed dataset is about 80000

In [26]:
max_tokens = 70000

vectorizer = tf.keras.layers.TextVectorization(max_tokens=70000,
                                               output_sequence_length=55)

next step is to adapt our vectorizer

In [27]:
vectorizer.adapt(train_sentence)

In [28]:
embedding_1  = tf.keras.layers.Embedding(input_dim = max_tokens,
                                         output_dim = 128,
                                         mask_zero = True,
                                         name="kiri")

In [29]:
train_label.shape

(2211861,)

Now,we should create our labels,in one hot encoded format

In [30]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_label.values.reshape(-1,1))
y_train_encoded = label_encoder.transform(train_label.values.reshape(-1,1))
y_test_encoded = label_encoder.transform(test_label.values.reshape(-1,1))
y_val_encoded = label_encoder.transform(val_label.values.reshape(-1,1))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [31]:
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder()

oneHotEncoder.fit(y_train_encoded.reshape(-1,1))
y_train_one_hot = oneHotEncoder.transform(y_train_encoded.reshape(-1,1)).toarray()
y_test_one_hot = oneHotEncoder.transform(y_test_encoded.reshape(-1,1)).toarray()
y_val_one_hot = oneHotEncoder.transform(y_val_encoded.reshape(-1,1)).toarray()

In [32]:
y_train_one_hot[2]

array([0., 0., 1., 0., 0.])

In [33]:
y_train_one_hot = np.array(y_train_one_hot)
y_test_one_hot = np.array(y_test_one_hot)
y_val_one_hot = np.array(y_val_one_hot)

In [34]:
train_label,y_train_one_hot

(0          BACKGROUND
 1          BACKGROUND
 2             METHODS
 3             METHODS
 4             METHODS
               ...    
 2211856       METHODS
 2211857       METHODS
 2211858       RESULTS
 2211859       RESULTS
 2211860       RESULTS
 Name: target, Length: 2211861, dtype: object,
 array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]]))

Now,lets create our datasets

In [35]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentence,y_train_one_hot))
val_dataset = tf.data.Dataset.from_tensor_slices((val_sentence,y_val_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_sentence,y_test_one_hot))

In [36]:
train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [37]:
y_train_one_hot[100]

array([0., 0., 0., 0., 1.])

and now,it is time to create our model

In [ ]:
input_1  = tf.keras.layers.Input(shape=(1,),dtype="string")
x = vectorizer(input_1)
x = embedding_1(x)
x = tf.keras.layers.Conv1D(128,padding="same",activation=tf.keras.activations.relu,kernel_size=2)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(32,activation = tf.keras.activations.relu)(x)
output_1 = tf.keras.layers.Dense(5,activation = tf.keras.activations.softmax)(x)

model_1 = tf.keras.Model(input_1,output_1)

In [ ]:
from tensorflow import keras

model_1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
hist1 = model_1.fit(train_dataset,
           epochs=4,
           verbose=1,
           validation_data = val_dataset,
                    validation_steps = int(0.05*len(val_dataset)))

Epoch 1/4
138242/138242 [==============================] - 895s 6ms/step - loss: 0.4382 - accuracy: 0.8402 - val_loss: 0.4208 - val_accuracy: 0.8438
Epoch 2/4
138242/138242 [==============================] - 899s 7ms/step - loss: 0.3901 - accuracy: 0.8583 - val_loss: 0.4093 - val_accuracy: 0.8500
Epoch 3/4
138242/138242 [==============================] - 906s 7ms/step - loss: 0.3655 - accuracy: 0.8678 - val_loss: 0.4102 - val_accuracy: 0.8486
Epoch 4/4
138242/138242 [==============================] - 979s 7ms/step - loss: 0.3465 - accuracy: 0.8755 - val_loss: 0.4251 - val_accuracy: 0.8438


In [ ]:
model_1_pred = model_1.evaluate(test_dataset)

922/922 [==============================] - 4s 4ms/step - loss: 0.4200 - accuracy: 0.8526


In [ ]:
model_1_pred_prob = model_1.predict(test_dataset)

922/922 [==============================] - 3s 3ms/step


In [ ]:
model_1_pred_prob = tf.argmax(model_1_pred_prob,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
score_model_1 = accuracy_score(model_1_pred_prob,y_test_encoded)

In [ ]:
score_model_1

0.8526430000339064

In [ ]:
model_1_pred_prob[:10],y_test_encoded[:10]

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 2, 4, 2, 2, 4, 4, 4])>,
 array([0, 0, 0, 0, 2, 2, 2, 4, 4, 4]))

as we can see,the accuracy is 85 percent which is very good i think
but lets make out model a little more complex to see if we can see any improvement

In [38]:
input_2 = tf.keras.layers.Input(shape=(1,), dtype="string")
x = vectorizer(input_2)
x = embedding_1(x)

x = tf.keras.layers.Conv1D(128,padding="same",activation = tf.keras.activations.relu,kernel_size=2)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.expand_dims(x, axis=1)
x = tf.keras.layers.LSTM(32,dropout=0.1,activation=tf.keras.activations.tanh,recurrent_dropout=0.1)(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dense(32)(x)
x = tf.keras.layers.LeakyReLU(0.1)(x)
x = tf.keras.layers.Dropout(0.1)(x)
output_2 = tf.keras.layers.Dense(5,activation = tf.keras.activations.softmax)(x)

model_2 = tf.keras.Model(input_2,output_2)

In [40]:

model_2.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
hist2 = model_2.fit(train_dataset,
           epochs=4,
           verbose=1,
           validation_data = val_dataset,
                    validation_steps = int(0.05*len(val_dataset)))

Epoch 1/4
138242/138242 [==============================] - 1925s 14ms/step - loss: 0.4667 - accuracy: 0.8331 - val_loss: 0.4274 - val_accuracy: 0.8396
Epoch 2/4
138242/138242 [==============================] - 1915s 14ms/step - loss: 0.4040 - accuracy: 0.8560 - val_loss: 0.4280 - val_accuracy: 0.8431
Epoch 3/4
137197/138242 [============================>.] - ETA: 14s - loss: 0.3793 - accuracy: 0.8653

In [ ]:
print("zaza")

In [ ]:
pd.DataFrame(hist2.history).plot()